# baseline v3

이 베이스라인 코드는 `사전학습 모델 로드`, `배치 학습`, `파인튜닝`, `양자화`, `PEFT` 등이 적용된 버전입니다.

Colab의 GPU 환경에서 개발되었습니다.
- 런타임 - 런타임 유형 변경 - GPU로 변경(T4 GPU 등)



# 환경 준비

개발 환경에 필요한 라이브러리 버전을 고정하고 최신 버전으로 라이브러리를 업데이트합니다.

- 아래 셀 실행
- 실행 완료 후 런타임 - 세션 다시 시작

In [1]:
!pip -q install "transformers>=4.44.2" "accelerate>=0.34.2" "peft>=0.13.2" "bitsandbytes>=0.43.1" datasets pillow pandas torch torchvision --upgrade

# 데이터 준비

개발에 필요한 데이터를 준비합니다.

- train.csv, train 폴더
- test.csv, test 폴더
- sample_submission.csv

본 베이스라인은 colab에서 구글 드라이브를 마운트하여 사용합니다.

데이터를 압축 해제하는데 몇 분 정도의 시간이 소요됩니다.

#### 실습 참고 내용

    챕터 2-2 합성 데이터 실습
    - 구글 드라이브 마운트 : drive()

In [2]:
!sudo chown -R team020:team020 /opt/hf-cache

In [3]:
# 압축 해제
!unzip "/home/team020/data.zip" -d "/content/"

Archive:  /home/team020/data.zip
checkdir:  cannot create extraction directory: /content
           Permission denied


# 라이브러리, 데이터, 설정

In [4]:
import os, re, math, random
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import torch
from typing import Dict, List, Any
from transformers import (
    AutoModelForVision2Seq,
    AutoProcessor,
    BitsAndBytesConfig,
    get_linear_schedule_with_warmup
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from tqdm import tqdm

# 이미지 로드 시 픽셀 제한 해제
Image.MAX_IMAGE_PIXELS = None

# 디바이스 GPU 우선 사용 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# 사전 학습 모델 정의
MODEL_ID = "Qwen/Qwen2.5-VL-3B-Instruct"
IMAGE_SIZE = 448 # 384
MAX_NEW_TOKENS = 1 # 8
SEED = 42 # 42
random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# 데이터셋 로드
# train_df = pd.read_csv("/home/team020/train.csv")
# test_df  = pd.read_csv("/home/team020/test.csv")

DATA_DIR   = "/home/team020"
TRAIN_CSV  = f"{DATA_DIR}/train.csv"
TEST_CSV   = f"{DATA_DIR}/test.csv"
CACHE_DIR  = f"{DATA_DIR}/.cache"  # 이미지/전처리 중간산출물 저장 용도(선택)

train_df  = pd.read_csv(TRAIN_CSV)
test_df   = pd.read_csv(TEST_CSV)

# 학습데이터 200개만 추출
# train_df = train_df.sample(n=200, random_state=SEED).reset_index(drop=True) # 200

/home/team020/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda


# 모델, Processor

7.5GB 정도의 모델 다운로드가 진행됩니다. 10~20분 정도가 소요됩니다.

#### 실습 참고 내용

    챕터 5-1 PEFT(파라미터 효율적 튜닝)
    - LoRA 구현 : LoraConfig()

In [5]:
# 양자화
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4", # nf4
    bnb_4bit_compute_dtype=torch.bfloat16, # torch.float16
)

# 프로세서
processor = AutoProcessor.from_pretrained(
    MODEL_ID,
    min_pixels=IMAGE_SIZE*IMAGE_SIZE,
    max_pixels=IMAGE_SIZE*IMAGE_SIZE,
    trust_remote_code=True,
)

# 사전학습 모델
base_model = AutoModelForVision2Seq.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# 양자화 모델로 로드
base_model = prepare_model_for_kbit_training(base_model)
base_model.gradient_checkpointing_enable()

# LoRA 세팅
lora_config = LoraConfig(
    r=32, # 8
    lora_alpha=64, # 16
    lora_dropout=0.10, # 0.05
    bias="none",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM",
)

# PEFT 모델 생성
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.
/home/team020/.venv/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:2284: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:28<00:00, 14.26s/it]


trainable params: 74,305,536 || all params: 3,828,928,512 || trainable%: 1.9406


# 프롬프트 템플릿

#### 실습 참고 내용

    챕터 5-1 PEFT(파라미터 효율적 튜닝)
    - 프롬프트 템플릿 : convert_to_chatml(), formatting_prompts_func()

In [6]:
# 모델 지시사항
SYSTEM_INSTRUCT = (
    # "You are a helpful visual question answering assistant. "
    # "Answer using exactly one letter among a, b, c, or d. No explanation."
    "You are a helpful visual QA assistant for multiple-choice. "
    "Reply with exactly ONE lowercase letter: a, b, c, or d. "
    "No punctuation, no newline, no extra text. If unsure, guess."
)

# 프롬프트
def build_mc_prompt(question, a, b, c, d):
    return (
        f"{question}\n"
        f"(a) {a}\n(b) {b}\n(c) {c}\n(d) {d}\n\n"
        # "정답을 반드시 a, b, c, d 중 하나의 소문자 한 글자로만 출력하세요."
        "정답은 반드시 a/b/c/d 중 소문자 1글자만, 마침표/개행 없이 출력하세요."
    )

# Custom Dataset, Collator

#### 실습 참고 내용

    챕터 1-2 MLP 구현
    - TensorDataset()

    챕터 5-2 데이터 생성 및 파인튜닝 (향후 학습 분량)
    - IntentDataset()

In [7]:
# 커스텀 데이터셋
class VQAMCDataset(Dataset):
    def __init__(self, df, processor, train=True):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.train = train

    def __len__(self): return len(self.df)

    def __getitem__(self, i):
        row = self.df.iloc[i]
        img = Image.open(row["path"]).convert("RGB")

        q = str(row["question"])
        a, b, c, d = str(row["a"]), str(row["b"]), str(row["c"]), str(row["d"])
        user_text = build_mc_prompt(q, a, b, c, d)

        messages = [
            {"role":"system","content":[{"type":"text","text":SYSTEM_INSTRUCT}]},
            {"role":"user","content":[
                {"type":"image","image":img},
                {"type":"text","text":user_text}
            ]}
        ]
        if self.train:
            gold = str(row["answer"]).strip().lower()
            messages.append({"role":"assistant","content":[{"type":"text","text":gold}]})

        return {"messages": messages, "image": img}

# 데이터 콜레이터
@dataclass
class DataCollator:
    processor: Any
    train: bool = True

    def __call__(self, batch):
        texts, images = [], []
        for sample in batch:
            messages = sample["messages"]
            img = sample["image"]

            text = self.processor.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=False
            )
            texts.append(text)
            images.append(img)

        enc = self.processor(
            text=texts,
            images=images,
            padding=True,
            return_tensors="pt"
        )

        # if self.train:
        #     enc["labels"] = enc["input_ids"].clone()
        
        if self.train:
            # full(assistant 포함) 렌더
            full_texts = texts
            # no-assistant(정답 제거) 렌더: 마지막 assistant turn 제거
            no_assist_texts = []
            for msgs in [s["messages"] for s in batch]:
                no_assist = msgs[:-1]  # 마지막 assistant gold 제거
                t = self.processor.apply_chat_template(no_assist, tokenize=False, add_generation_prompt=False)
                no_assist_texts.append(t)

            # full_ids = self.processor(text=full_texts, images=images, padding=True, return_tensors="pt")
            # no_assist_ids = self.processor(text=no_assist_texts, images=images, padding=True, return_tensors="pt")

            # enc = full_ids  # enc를 full로 교체
            # enc["labels"] = enc["input_ids"].clone()

            # # no-assistant 길이 이전은 모두 마스킹(-100)
            # for i in range(enc["labels"].size(0)):
            #     cutoff = (no_assist_ids["input_ids"][i] != self.processor.tokenizer.pad_token_id).sum()
            #     enc["labels"][i, :cutoff] = -100

            enc_full = self.processor(text=full_texts, images=images, padding=True, return_tensors="pt")
            enc_mask = self.processor(text=no_assist_texts, images=images, padding=True, return_tensors="pt")
            enc = enc_full
            enc["labels"] = enc_full["input_ids"].clone()
            pad_id = self.processor.tokenizer.pad_token_id
            for i in range(enc["labels"].size(0)):
                cutoff = (enc_mask["input_ids"][i] != pad_id).sum()
                enc["labels"][i, :cutoff] = -100

        return enc


# DataLoader

#### 실습 참고 내용

    챕터 3-1 Transfer Learning 기반의 CNN 모델 학습
    - 데이터로더 정의 : DataLoader()

In [8]:
!pip install -U scikit-learn


In [9]:
# 검증용 데이터 분리
# split = int(len(train_df)*0.9) # 0.9
# train_subset, valid_subset = train_df.iloc[:split], train_df.iloc[split:]

from sklearn.model_selection import train_test_split
train_subset, valid_subset = train_test_split(
    train_df, test_size=0.1, random_state=SEED, stratify=train_df["answer"]
)
train_subset = train_subset.reset_index(drop=True)
valid_subset = valid_subset.reset_index(drop=True)


# VQAMCDataset 형태로 변환
train_ds = VQAMCDataset(train_subset, processor, train=True)
valid_ds = VQAMCDataset(valid_subset, processor, train=True)

# 데이터로더
# train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, collate_fn=DataCollator(processor, True), num_workers=0) # batch_size=1
# valid_loader = DataLoader(valid_ds, batch_size=1, shuffle=False, collate_fn=DataCollator(processor, True), num_workers=0) # batch_size=1
# train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, collate_fn=DataCollator(processor, True), num_workers=4, pin_memory=True) # batch_size=1
# valid_loader = DataLoader(valid_ds, batch_size=4, shuffle=False, collate_fn=DataCollator(processor, True), num_workers=4, pin_memory=True) # batch_size=1

train_loader = DataLoader(
    train_ds, batch_size=2, shuffle=True,
    collate_fn=DataCollator(processor, True),
    num_workers=4,                # BEFORE: 0 → CPU 디코딩 병렬화
    pin_memory=True,
    persistent_workers=True,      # 워커 재사용
    prefetch_factor=4             # 배치 선로딩
)
valid_loader = DataLoader(
    valid_ds, batch_size=2, shuffle=False,
    collate_fn=DataCollator(processor, True),
    num_workers=4, pin_memory=True,
    persistent_workers=True, prefetch_factor=4
)




# fine-tuning

- 200개만 학습 : 10~20분 소요

#### 실습 참고 내용

    챕터 1-2 MLP 구현
    - 모델 정의 : SimpleMLP(), SequentialMLP()

    챕터 3-1 Transfer Learning 기반의 CNN 모델 학습
    - 학습 루프 : 문제 6: 모델 학습을 위한 반복문
    - 추론 : with torch.no_grad(), model.eval()

In [10]:
from tqdm.auto import tqdm

model = model.to(device)
GRAD_ACCUM = 4 # 4

# 옵티마이저, 학습 스케줄러
optimizer = torch.optim.AdamW(model.parameters(), lr=2.5e-4, weight_decay=0.01, betas=(0.9, 0.999)) # 1e-4
# num_training_steps = 1 * math.ceil(len(train_loader)/GRAD_ACCUM)
# scheduler = get_linear_schedule_with_warmup(optimizer, int(num_training_steps*0.03), num_training_steps) # 0.03
EPOCHS = 5
steps_per_epoch = math.ceil(len(train_loader)/GRAD_ACCUM)
num_training_steps = EPOCHS * steps_per_epoch
warmup_steps = int(num_training_steps * 0.1) # 0.06

# scheduler = get_linear_schedule_with_warmup(optimizer, warmup_steps, num_training_steps)
from transformers import get_cosine_schedule_with_warmup
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=num_training_steps)

# 스케일러
# scaler = torch.cuda.amp.GradScaler(enabled=True)
scaler = torch.cuda.amp.GradScaler(enabled=False)
model.config.use_cache = False

# 학습 루프
global_step = 0
# for epoch in range(3): # 1
for epoch in range(EPOCHS): # 1
    running = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1} [train]", unit="batch")
    for step, batch in enumerate(progress_bar, start=1):
        batch = {k:v.to(device) for k,v in batch.items()}

        with torch.cuda.amp.autocast(dtype=torch.bfloat16):
            outputs = model(**batch)
            loss = outputs.loss / GRAD_ACCUM

        loss.backward()                           # ← 단 한 번만 호출
        running += loss.item()

        if step % GRAD_ACCUM == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()

            avg_loss = running                    # GRAD_ACCUM으로 나눈 loss를 누적했으니 그대로 평균
            progress_bar.set_postfix({"loss": f"{avg_loss:.3f}"})
            running = 0.0

    model.eval()
    val_loss = 0.0
    val_steps = 0
    with torch.no_grad(), torch.cuda.amp.autocast(dtype=torch.bfloat16):
        for vb in tqdm(valid_loader, desc=f"Epoch {epoch+1} [valid]", unit="batch"):
            vb = {k:v.to(device) for k,v in vb.items()}
            val_loss += model(**vb).loss.item()
            val_steps += 1
    print(f"[Epoch {epoch+1}] valid loss {val_loss/val_steps:.4f}")
    model.train()

# 모델 저장
SAVE_DIR = "/home/team020/qwen2_5_vl_3b_lora" # Qwen/Qwen2.5-VL-3B-Instruct-AWQ
model.save_pretrained(SAVE_DIR)
processor.save_pretrained(SAVE_DIR)
print("Saved:", SAVE_DIR)


/tmp/ipykernel_6332/272494740.py:21: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=False)
Epoch 1 [train]:   0%|          | 0/1749 [00:00<?, ?batch/s]

/tmp/ipykernel_6332/272494740.py:33: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):
Epoch 1 [train]: 100%|██████████| 1749/1749 [31:55<00:00,  1.10s/batch, loss=0.019]
/tmp/ipykernel_6332/272494740.py:53: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(dtype=torch.bfloat16):
Epoch 1 [valid]: 100%|██████████| 195/195 [01:39<00:00,  1.96batch/s]


[Epoch 1] valid loss 0.0514


Epoch 2 [train]:   0%|          | 0/1749 [00:00<?, ?batch/s]/home/team020/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Epoch 2 [valid]: 100%|██████████| 195/195 [01:38<00:00,  1.99batch/s]


[Epoch 2] valid loss 0.0529


Epoch 3 [valid]: 100%|██████████| 195/195 [01:37<00:00,  1.99batch/s]


[Epoch 3] valid loss 0.0447


Epoch 4 [train]:   9%|▉         | 157/1749 [02:40<27:10,  1.02s/batch, loss=0.014]


KeyboardInterrupt: 

# inference

30분~1시간 소요

#### 실습 참고 내용

    챕터4-1 RAG 기반 Customer Service AI 에이전트 개발
    - 데이터 파서 : langchain_core.output_parsers(), StrOutputParser()

    챕터 3-1 Transfer Learning 기반의 CNN 모델 학습
    - 학습 루프 : 문제 6: 모델 학습을 위한 반복문
    - 추론 : with torch.no_grad(), model.eval()

In [ ]:
# 데이터 파서 : 모델의 응답에서 선지를 추출
def extract_choice(text: str) -> str:
    text = text.strip().lower()

    lines = [l.strip() for l in text.splitlines() if l.strip()]
    if not lines:
        return "a"
    last = lines[-1]
    if last in ["a", "b", "c", "d"]:
        return last

    tokens = last.split()
    for tok in tokens:
        if tok in ["a", "b", "c", "d"]:
            return tok
    return "a"

# 추론을 위해 모든 레이어 활성화
model.eval()
preds = []

# 추론 루프
for i in tqdm(range(len(test_df)), desc="Inference", unit="sample"):
    row = test_df.iloc[i]
    img = Image.open(row["path"]).convert("RGB")
    user_text = build_mc_prompt(row["question"], row["a"], row["b"], row["c"], row["d"])

    messages = [
        {"role":"system","content":[{"type":"text","text":SYSTEM_INSTRUCT}]},
        {"role":"user","content":[
            {"type":"image","image":img},
            {"type":"text","text":user_text}
        ]}
    ]

    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[img], return_tensors="pt").to(device)

    with torch.no_grad():
        # out_ids = model.generate(**inputs, max_new_tokens=1, do_sample=False,
        #                          eos_token_id=processor.tokenizer.eos_token_id)
        # out_ids = model.generate(**inputs, max_new_tokens=2, do_sample=False,
        #                          eos_token_id=processor.tokenizer.eos_token_id,
        #                          pad_token_id=processor.tokenizer.pad_token_id)
        bad = []
        vocab = processor.tokenizer.get_vocab()
        allow = set(processor.tokenizer.convert_tokens_to_ids(list("abcd")))
        for tok, tid in vocab.items():
            if len(tok) == 1 and tok.isalpha() and tid not in allow:
                bad.append([tid])

        out_ids = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,      # ← 상단 상수와 일관
            do_sample=False,
            eos_token_id=processor.tokenizer.eos_token_id,
            pad_token_id=processor.tokenizer.pad_token_id,
            bad_words_ids=bad                    # 선택: 모델/토크나이저에 따라 무시될 수도 있음
        )

    output_text = processor.batch_decode(out_ids, skip_special_tokens=True)[0]
    # print("output_text:", output_text)
    # print("extract_choice:", extract_choice(output_text))
    preds.append(extract_choice(output_text))

# 제출 파일 생성
submission = pd.DataFrame({"id": test_df["id"], "answer": preds})
submission.to_csv("/home/team020/submission.csv", index=False)
print("Saved /home/team020/submission.csv")

Inference:   0%|          | 0/3887 [00:00<?, ?sample/s]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Inference: 100%|██████████| 3887/3887 [35:03<00:00,  1.85sample/s]

Saved /home/team020/submission.csv


In [ ]:
# 모델 응답 예시
print(output_text)

system
You are a helpful visual question answering assistant. Answer using exactly one letter among a, b, c, or d. No explanation.
user
이 사진의 주요 상황은 무엇인가요?
(a) 수업 시간에 공부하고 있다
(b) 회의에 참석하고 있다
(c) 졸업식 준비 중이다
(d) 시험을 치르고 있다

정답을 반드시 a, b, c, d 중 하나의 소문자 한 글자로만 출력하세요.
assistant
c
